In [2]:
import pandas as pd
import math



In [3]:
# Replace 'your_file.csv' with the path to your CSV file
file_path = './datathon_2024_dataset.csv'

# Read the CSV file into a pandas DataFrame
original_df = pd.read_csv(file_path)

print(original_df.head())

   Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
0           0   20000329       NYN       CHN        False           3   
1           1   20000330       CHN       NYN        False           1   
2           2   20000403       ATL       COL         True           2   
3           3   20000403       CIN       MIL         True           3   
4           4   20000403       FLO       SFN        False           6   

   away_score  venue        venue_name        city  ... home_hbp  away_pa  \
0           5  TOK01        Tokyo Dome       Tokyo  ...        1       38   
1           5  TOK01        Tokyo Dome       Tokyo  ...        1       44   
2           0  ATL02      Turner Field     Atlanta  ...        0       32   
3           3  CIN08     Cinergy Field  Cincinnati  ...        0       20   
4           4  MIA01  Sun Life Stadium       Miami  ...        0       38   

   away_1b  away_2b  away_3b  away_hr  away_fo  away_so  away_bb  away_hbp  
0        5        1  

In [4]:
location_file_path = './cities_coordinates.xlsx'
location_df = pd.read_excel(location_file_path)
print(location_df.head())

         city  latitude  longitude
0     Atlanta   33.7490   -84.3880
1  Cincinnati   39.1031   -84.5120
2       Miami   25.7617   -80.1918
3    Montreal   45.5017   -73.5673
4    New York   40.7128   -74.0060


In [5]:
# Calculat the distance between two points.

def haversine(lat1, lon1, lat2, lon2):
    # 将经纬度转换为弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # 地球半径（单位：公里）
    radius = 6371.0

    # 计算距离
    distance = radius * c

    return distance

# 示例：两个地点的经纬度
lat1, lon1 = 37.7749, -122.4194  # 地点1（例如，旧金山）
lat2, lon2 = 34.0522, -118.2437  # 地点2（例如，洛杉矶）

# 计算距离
result = haversine(lat1, lon1, lat2, lon2)
print(f"distance is：{result:.2f} ")


distance is：559.12 


In [6]:
#print out all city names

# Delelete all rows with city "Tokyo", "London", "Sydney",  "San Juan", "Monterrey"
original_df = original_df[original_df['city'] != 'Tokyo']
original_df = original_df[original_df['city'] != 'London']
original_df = original_df[original_df['city'] != 'Sydney']
original_df = original_df[original_df['city'] != 'San Juan']
original_df = original_df[original_df['city'] != 'Monterrey']

print(original_df['city'].unique())

# Count the number of unique city
print(original_df['city'].nunique())



['Atlanta' 'Cincinnati' 'Miami' 'Montreal' 'New York' 'St. Louis'
 'Anaheim' 'Baltimore' 'Minneapolis' 'Oakland' 'Arlington' 'Toronto'
 'Phoenix' 'Pittsburgh' 'Seattle' 'Houston' 'Kansas City' 'St. Petersburg'
 'Chicago' 'Denver' 'Milwaukee' 'Philadelphia' 'San Diego' 'San Francisco'
 'Boston' 'Detroit' 'Los Angeles' 'Cleveland' 'Washington'
 'Lake Buena Vista' 'Fort Bragg' 'Williamsport' 'Omaha' 'Buffalo'
 'Dunedin' 'Dyersville']
36


In [7]:
import pandas as pd

# Assuming df1 is the 30x3 table and df2 is the 5000xN table
# Replace these with your actual DataFrames

# Assuming both venue_name columns are of the same data type


# Merge the two DataFrames based on 'venue_name'
merged_df = pd.merge(original_df, location_df, on='city', how='inner')

# 'inner' will keep only the rows where venue_name exists in both tables

# Display the merged DataFrame

# Sort merged_df by game_date
merged_df = merged_df.sort_values(by='game_date')
print(merged_df.head(20))



       Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
0               2   20000403       ATL       COL         True           2   
11692           8   20000403       ANA       NYA        False           2   
13586           9   20000403       BAL       CLE         True           1   
15476          10   20000403       MIN       TBA        False           0   
6017            6   20000403       NYN       SDN         True           2   
5656            5   20000403       MON       LAN        False           4   
17369          11   20000403       OAK       DET        False           4   
19257          12   20000403       TEX       CHA         True          10   
21152          13   20000403       TOR       KCA         True           5   
3781            4   20000403       FLO       SFN        False           6   
1891            3   20000403       CIN       MIL         True           3   
9802            7   20000403       SLN       CHN         True           7   

In [14]:
'''# Add two columns: "team name: distance". 
# The distance is calculated by the distance between this row and the previous row with the same team name(either home or away team). If there is no row above with the same team name, the distance is 0.

# Create a new column 'distance' and set it to 0
merged_df['home_team_distance'] = 0

# Iterate through each row in the DataFrame
for index, row in merged_df.iterrows():
    # If this is not the first row
    if index > 0:
        # If the current row's team name is the same as the previous row's team name
        if row['home_team'] == merged_df.loc[index - 1, 'home_team'] or row['home_team'] == merged_df.loc[index - 1, 'away_team']:
            # Calculate the distance between the two rows
            distance = haversine(row['latitude'], row['longitude'], merged_df.loc[index - 1, 'latitude'], merged_df.loc[index - 1, 'longitude'])
            # Set the current row's distance to the calculated distance
            merged_df.loc[index, 'home_team_distance'] = distance

# Display the DataFrame
print(merged_df.head(20))'''

# Initialize the distance columns
merged_df['home_team_distance'] = 0
merged_df['away_team_distance'] = 0

# Dictionary to keep track of the last location of each team
last_locations = {}

# Iterate through each row in the DataFrame
for index, row in merged_df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    current_location = (row['latitude'], row['longitude'])
    
    # For home team
    if home_team in last_locations:
        # Calculate the distance from the last location
        last_location = last_locations[home_team]
        distance = haversine(current_location[0], current_location[1], last_location[0], last_location[1])
        merged_df.at[index, 'home_team_distance'] = distance
    last_locations[home_team] = current_location

    # For away team
    if away_team in last_locations:
        # Calculate the distance from the last location
        last_location = last_locations[away_team]
        distance = haversine(current_location[0], current_location[1], last_location[0], last_location[1])
        merged_df.at[index, 'away_team_distance'] = distance
    last_locations[away_team] = current_location

# Display the DataFrame
print(merged_df.head(50))



       Unnamed: 0  game_date home_team away_team  is_day_game  home_score  \
0               2   20000403       ATL       COL         True           2   
11692           8   20000403       ANA       NYA        False           2   
13586           9   20000403       BAL       CLE         True           1   
15476          10   20000403       MIN       TBA        False           0   
6017            6   20000403       NYN       SDN         True           2   
5656            5   20000403       MON       LAN        False           4   
17369          11   20000403       OAK       DET        False           4   
19257          12   20000403       TEX       CHA         True          10   
21152          13   20000403       TOR       KCA         True           5   
3781            4   20000403       FLO       SFN        False           6   
1891            3   20000403       CIN       MIL         True           3   
9802            7   20000403       SLN       CHN         True           7   